# Build a language model based on wikipedia in you language 

The notebooke includes the whole process but you will need to help the process if the following problem happens:  
-you internet connection is interrupted (stage 1)

-you run out of diskspace

-because of the huge memory consumption. creatig the databunch for the training requires lots of memory


In order ot handle these problemn the notebook is divided into stages. If the process is failes in a stage then you can resume the processing from the beginning of that stage by:
-restarting the kernel "Kernel"/Restart & Clear Output

-running the cells in stage 0: initialization

-running the cells from the start of the stage that failed 

# Stage 0: Initialisation
lang: You must set the parameter "lang" to the language you want to build a model for. Fx:

fr: for french

en: for english

de:for german

da: for danish 

etc.

pathData: You must se the location where you want your data stored using the parameter pathData. Consider using a ssd-rive

In [1]:
lang="en"

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from fastai.text import * 

from IPython.display import display, HTML
from pathlib import *
import re
import shutil

In [4]:
pathData       = Path("../../data/nlp")
path           = pathData / lang
pathDump       = path/"wiki-dump"
pathJson       = path/"wiki-json"

pathTrainValid = path/"wiki-train_valid"
pathTxt        = pathTrainValid/"txt"
pathToks       = pathTrainValid/"toks"
pathcsv        = pathTrainValid/"wiki.csv"

cache_name   = "sp-model"
pathVocab    = pathTrainValid / cache_name

# requried libraries: 
conda install -c anaconda psutil 

conda install -c anaconda git 

#conda install -c menpo wget 
conda install curl


# Stage 1: download the selected language from wikipedia and convert the articles to json

In [5]:
if len(list(pathDump.glob("*.bz2")))==0:
    pathDump.mkdir(parents=True,exist_ok=True)
    #fn  = f"{lang}wiki-latest-pages-articles.xml.bz2"
    fn  = f"{lang}wiki-latest-pages-articles-multistream.xml.bz2"
    url = f"https://dumps.wikimedia.org/{lang}wiki/latest/{fn}"
    #cmd = f"wget -c --no-check-certificate --show-progress {str(url)} -P {str(pathDump)}"
    cmd = f'curl -k -C - -o "{str(pathDump/fn)}" "{str(url)}"'
    print(f"If the command fails in the notebook then copy the command and run it in the terminal:{cmd}")
    ! $cmd

# Stage 2: Convert wikipedia dump to articles in json

In [6]:
pathWikiExtractor = Path("../wikiextractor")

In [7]:
if not pathJson.exists():
    !git clone https://github.com/attardi/wikiextractor.git $pathWikiExtractor
    cmd = f"cd {str(pathWikiExtractor)} && python setup.py install"
    ! $cmd

In [8]:
%%time
if not pathJson.exists():
    #extracting the json-files. This takes about 1 hour for french with files read from and saved to a ssd hardrive 
    fn_wikidump = list(pathDump.iterdir())[0]
    #cmd = f"cd {str(pathWikiExtractor)} && python WikiExtractor.py -o {str(pathJson)} --json --processes {defaults.cpus} -q --filter_disambig_pages --no-templates --discard_elements gallery,timeline,noinclude {str(fn_wikidump)}"
    #cmd = f"cd {str(pathWikiExtractor)} && python WikiExtractor.py -o {str(pathJson)} --json --processes {defaults.cpus} -q --filter_disambig_pages --discard_elements gallery,timeline,noinclude {str(fn_wikidump)}"
    cmd = f"cd {str(pathWikiExtractor)} && python WikiExtractor.py -o {str(pathJson)} --json --processes {defaults.cpus} -q --filter_disambig_pages --no-templates --discard_elements gallery,timeline,noinclude {str(fn_wikidump)}"
    print(f"If WikiExtractor fails in the notebook then copy the command and run it in the terminal:{cmd}")
    ! $cmd

Wall time: 0 ns


# Stage 3: building a vocabulary using sentencepiece

Now we separate the title of the wikipedia section from text section that we keep. 

In order to makes a first reduction on the number of section we clean the text with the preprocessing rules from fastai and ignore text with less than "minWords"

You must set the lenght of the shortes sections to keep using the parameter "minWords"

In [9]:
minWords  = 1
chunksize = 25000

In [10]:
!pip install sentencepiece

In [11]:
from fastai_sentencepiece import *
from filetokenizer import *

In [12]:
data = None
if not pathTxt.exists():
    swm = SentencepieceWikiVocab(lang=lang, pathJson=pathJson, pathcsv=pathcsv, pathTxt=pathTxt, pathVocab=pathVocab)    
    %time swm.wikijson2TrainingData()
    %time data = pd.read_csv(pathcsv)
    gc.collect()

In [13]:
if data is not None:
    percentiles = 0.01 * np.asarray([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 
                                     1,2,3,4,10,20, 50, 75, 90, 95, 96, 97, 98, 
                                     99, 99.1, 99.2, 99.3, 99.4, 99.5, 99.6, 99.7, 99.8, 99.9,99.99]) 
    %time display(data.describe(percentiles,include=[np.number]))
    print(f"data:{data.shape}")

In [14]:
if data is not None:
    lpl = data.alpha_r3.values
    #ix = lpl<8
    ix = np.logical_and(lpl>.17,lpl<.18)
    data_empty = data[ix]
    print(f"data_empty:{data_empty.shape}")
    
    percentiles = 0.01 * np.asarray([1,10,25, 50, 75, 90, 99])     
    %time data_empty.describe(percentiles,include=[np.number])
    display(data_empty)
    print(data_empty.text.head().values)
    print(data_empty.text.tail().values)
    gc.collect()

In [15]:
if not (pathVocab/"m.model").exists():
    swm = SentencepieceWikiVocab(lang=lang, pathJson=pathJson, pathcsv=pathcsv, pathTxt=pathTxt, pathVocab=pathVocab)
    %time swm.trainVocabulary()

In [16]:
if not (pathVocab/"itos.pkl").exists():
    swm = SentencepieceWikiVocab(lang=lang, pathJson=pathJson, pathcsv=pathcsv, pathTxt=pathTxt, pathVocab=pathVocab)
    swm.convertSPVocab2FastaiVocab()

# Show some examples using the vocabulary

In [17]:
swm = SentencepieceWikiVocab(lang=lang, pathJson=pathJson, pathcsv=pathcsv, pathTxt=pathTxt, pathVocab=pathVocab)
sp  = spm.SentencePieceProcessor()
sp.Load(str(pathVocab / "m.model"))
print("\nSize of vocabulary:",sp.GetPieceSize())

control_symbols = ["<unk>","xxbos","xxeos","xxpad"]
print(f"Control symbol:")
for s in control_symbols: print(f"{s}({sp.PieceToId(s)})")

print(f"\nUser defined symbols:")
for s in swm.getUserdefinedSymbols():print(f"{s}({sp.PieceToId(s)})")

sentence = "She is tall. He is small".lower()
print(f"Sentence:          {sentence}")
print(f"Sentence as pieces:{sp.EncodeAsPieces(sentence)}")
print(f"Sentence as ids:   {sp.EncodeAsIds(sentence)}")
print(f"Sentence from ids:   {sp.DecodeIds(sp.EncodeAsIds(sentence))}")


Size of vocabulary: 4000
Control symbol:
<unk>(0)
xxbos(1)
xxeos(2)
xxpad(3)

User defined symbols:
xxfld(4)
xxmaj(5)
xxup(6)
xxrep(7)
xxwrep(8)
0(9)
1(10)
2(11)
3(12)
4(13)
5(14)
6(15)
7(16)
8(17)
9(18)
°(19)
%(20)
$(21)
§(22)
£(23)
€(24)
((25)
)(26)
<(27)
>(28)
"(29)
'(30)
“(31)
”(32)
‘(33)
’(34)
!(35)
,(57)
;(36)
:(37)
.(38)
•(39)
—(40)
|(41)
\(42)
/(43)
*(44)
+(45)
-(46)
=(47)
⁄(48)
′(49)
_(50)
#(51)
&(52)
?(53)
Sentence:          she is tall. he is small
Sentence as pieces:['▁she', '▁is', '▁tall', '.', '▁he', '▁is', '▁small']
Sentence as ids:   [126, 66, 3118, 38, 74, 66, 557]
Sentence from ids:   she is tall. he is small


# Prepare training of the language model

# LM Training
Set the minimum number of tokens for the sections that we shall retain

In [18]:
#Discard section with kess than "minTok" tokens
minToks = 10

In [ ]:
spt_func  = partial(SentencepieceTokenizer.create, pathVocab=pathVocab)
spt_func.__name__ = SentencepieceTokenizer.__name__
spt       = spt_func(lang="en")

unk_idx,pad_idx = spt.vocab().numericalize([text.transform.UNK])[0], spt.vocab().numericalize([text.transform.PAD])[0]
print(f"unk_idx:{unk_idx} pad_idx:{pad_idx}")
vocab,max_vocab = spt.vocab(), len(spt.vocab().itos)
trainTokenizer  = SPFileTokenizer(pathToks/"train", spt_func,"en",vocab,n_cpus=max(defaults.cpus-1,1))
validTokenizer  = SPFileTokenizer(pathToks/"valid", spt_func,"en",vocab,n_cpus=max(defaults.cpus-1,1))

unk_idx:0 pad_idx:3


In [ ]:
files   = np.asarray( list(pathTxt.glob("*.txt")) )
nrows   = len(files)
split   = 0.2
splitindex, index = int(nrows*split+.5), np.random.permutation(np.arange(nrows)) 

chunksize=0

trainList = TextList( files[:-splitindex], vocab=vocab, pad_idx=pad_idx, 
                      processor=[SPFileTokenizeProcessor(tokenizer=trainTokenizer, 
                                                       chunksize=chunksize, mark_fields=False)])

validList = TextList( files[-splitindex:], vocab=vocab, pad_idx=pad_idx, 
                      processor=[SPFileTokenizeProcessor(tokenizer=validTokenizer, 
                                                       chunksize=chunksize, mark_fields=False)])


In [ ]:
if not (pathToks/"train").exists():
    %time trainList.process()
    trainList=None
    gc.collect()    

In [ ]:
if not (pathToks/"valid").exists():
    %time p = validList.process()
    validList=None
    gc.collect()

# The following cells are used to find macUnk, minToks and maxToks.
The 4 cells were deactivate after deciding on these values

In [ ]:
%time trainIDS = trainTokenizer.getIds()
%time validIDS = validTokenizer.getIds()

Wall time: 1min 40s
Wall time: 30 s


In [ ]:
#how much do we loos by removing sentences with unks
unkPrSentence        = np.asarray([np.sum(s==unk_idx) for s in trainIDS],dtype=np.int)
percentUnkPrSentence = np.asarray([np.sum(s==unk_idx)/len(s) for s in trainIDS])
percent              = np.asarray([0, 50, 75, 90, 97, 98.5, 99, 99.1, 99.2, 99.3, 99.4, 99.5, 99.6, 99.7, 99.8, 99.9, 99.95,  99.99, 100]) 
percentiles          = np.percentile(unkPrSentence,percent).astype(np.int)
print("unkPrSentence %  percentile ")
for i in range(len(percent)):print(f"{percent[i]}  {percentiles[i]}")   

unkPrSentence=None
gc.collect()

unkPrSentence %  percentile 
0.0  0
50.0  0
75.0  0
90.0  0
97.0  0
98.5  0
99.0  0
99.1  1
99.2  1
99.3  1
99.4  1
99.5  1
99.6  2
99.7  2
99.8  3
99.9  4
99.95  6
99.99  11
100.0  133


7

In [ ]:
print(f"Traning:    number of arrays:{len(trainIDS)} - number of ids:{sum(len(ids) for ids in trainIDS)}")
print(f"Validation: number of arrays:{len(validIDS)} - number of ids:{sum(len(ids) for ids in validIDS)}")

#Analyse the distribution of the legnth of tokens sequences in the ragged/jagged array of tokes
sectionlengths = np.asarray([len(s) for s in trainIDS],dtype=np.int32)
plt.hist(sectionlengths[sectionlengths<500], 250, density=True, facecolor='g', alpha=0.75)
np.histogram(sectionlengths[sectionlengths<1000],50)

print(f"Lenght of token rags min:{min(sectionlengths)} max:{np.max(sectionlengths)} - median:{np.median(sectionlengths)}")
print(f"Lenght of token rags mean:{np.mean(sectionlengths)} std:{np.std(sectionlengths)}")

percent     = np.asarray([ 0, 0.11, 0.2, 1, 1.5, 2, 25,50,75,90,97,99,99.5,99.9,99.99,100]) # %np.arange(101,dtype=np.int)
percentiles = np.percentile(sectionlengths,percent).astype(np.int)
print("%  percentile")
for i in range(len(percent)):print(f"{percent[i]}  {percentiles[i]}")
minToks = percentiles[np.where( percent==0.11 )[0][0]]
maxToks = percentiles[np.where( percent==99.99 )[0][0]]
print(f"\nminToks:{minToks} maxToks:{maxToks}")
print(f"rags <= minToks:{np.sum(sectionlengths <= minToks)}")
print(f"rags >= maxToks:{np.sum(sectionlengths >= maxToks)}")

sectionlengths = trainIDS = validIDS = None
gc.collect()

Traning:    number of arrays:25685835 - number of ids:2980870303
Validation: number of arrays:6181342 - number of ids:722508809
Lenght of token rags min:7 max:2038 - median:94.0
Lenght of token rags mean:116.05113491541155 std:88.30266532778766
%  percentile
0.0  7
0.11  11
0.2  12
1.0  15
1.5  16
2.0  18
25.0  53
50.0  94
75.0  154
90.0  228
97.0  327
99.0  423
99.5  488
99.9  662
99.99  975
100.0  2038

minToks:11 maxToks:975
rags <= minToks:36511
rags >= maxToks:2589

In [ ]:
maxUnks,minToks,maxToks = 0,14,400

%time trainIDS = trainTokenizer.getIds(minToks, maxToks, maxUnks, unk_idx=unk_idx)
%time validIDS = validTokenizer.getIds(minToks, maxToks, maxUnks, unk_idx=unk_idx)

In [ ]:
print(f"Traning:    number of arrays:{len(trainIDS)} - number of ids:{sum(len(ids) for ids in trainIDS)}")
print(f"Validation: number of arrays:{len(validIDS)} - number of ids:{sum(len(ids) for ids in validIDS)}")
sectionlengths = np.asarray([len(s) for s in trainIDS],dtype=np.int32)
plt.hist(sectionlengths[sectionlengths<maxToks], 300, density=True, facecolor='g', alpha=0.75)
np.histogram(sectionlengths[sectionlengths<1000],50)
print(f"Lenght of token rags min:{min(sectionlengths)} max:{np.max(sectionlengths)} - median:{np.median(sectionlengths)}")
gc.collect()

In [ ]:
#shuffle the rags so that any subset is composed of similar text

In [ ]:
#from languagemodelloader import *

#nTrainToks, nValidToks = int(5e6),int(2e6)
nTrainToks, nValidToks = int(5e5),int(2e5)
#nTrainToks, nValidToks = int(5e4),int(1e4)
#nTrainToks, nValidToks = int(5e1),int(1e1)
if nTrainToks>0 and nValidToks>0:
    trainIDS_ = trainIDS[0:nTrainToks]
    validIDS_ = validIDS[0:nValidToks]
else:
    trainIDS_ = trainIDS
    validIDS_ = validIDS

#dblm = TextLMDataBunch.from_ids( pathTrainValid, vocab, trainIDS_, validIDS_, bptt=70, bs=32, num_workers=0, lengths=lengths)
%time dblm = TextLMDataBunch.from_ids( pathTrainValid, vocab, trainIDS_, validIDS_, bptt=90, bs=384, num_workers=0, no_check=True)
nTrainToks, nValidToks

In [ ]:
dblm.show_batch()

In [ ]:
%time learn = language_model_learner(dblm, drop_mult=1, qrnn=False, pad_token=pad_idx,  callback_fns=ShowGraph )

In [ ]:
gc.collect()

In [ ]:
%time learn.lr_find()

In [ ]:
gc.collect()

In [ ]:
learn.recorder.plot(skip_start=0, skip_end=12)

In [ ]:
gc.collect()

In [ ]:
#%load_ext line_profiler
#%time learn.fit_one_cycle(1,2e-3, moms=(0.8,0.7))
#%time learn.fit_one_cycle(1,4e-3, moms=(0.95,0.85))
#%time learn.fit_one_cycle(20,4e-3, moms=(0.8,0.7), wd=1e-7)
#learn.clip_grad(clip=0.12)
#%time learn.fit_one_cycle(1,5e-3, moms=(0.99,0.2))
epochs=10
#%time learn.fit_one_cycle(cyc_len=epochs, max_lr=5e-3, moms=(0.99,0.7))
%time learn.fit_one_cycle(cyc_len=epochs, max_lr=5e-3, moms=(0.8,0.7)) #256,384

In [ ]:
#remember to update the last validation loss to get your perplexity score
print( f"Perplexity: {exp(3.328701)}") #best: 3.221317
trainTokens={sum(len(s) for s in trainIDS_)}
print( f"Number of tokens in trainingdata:   {sum(len(s) for s in trainIDS_)}")
print( f"Number of tokens in validationdata: {sum(len(s) for s in validIDS_)}")

In [ ]:
learn.save(f'lm-32k-sentencepiece-{epochs}-{len(trainIDS_)}-{len(validIDS_)} dropout{int(round(10*drop_mult)} ')

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.recorder.plot_metrics()

In [ ]:
#TEXT = "if i knew back then what i know now," #  i would not agree # seen in polval
TEXT = "i liked this movie because"
#TEXT = "anarchy is a political philosophy that"
N_WORDS = 40
N_SENTENCES = 5

In [ ]:
print(("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES))).replace("▁",""))

In [ ]:
print(("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.25) for _ in range(N_SENTENCES))).replace("▁",""))